In [2]:
import evaluation

## Few shot prompting
Here we give in the prompt a sequence of unverified (role: user) and corresponding verified (role: assistant) programs. In the end we give an unverified program, and expect the assistant to generate a verified program.

### Run single
Modify the `EXAMPLE_TO_HOLD_OUT` variable in the cell below to see the output from GPT, and corresponding output from Nagini.

In [6]:
EXAMPLE_TO_HOLD_OUT = "append"
response = evaluation._call_gpt_with_timeout(
    model="gpt-3.5-turbo",
    messages=evaluation.get_few_shot_prompt(EXAMPLE_TO_HOLD_OUT, with_errors=True),
)
program_snippet = evaluation._print_and_process_response(response)
print("Verification result:\n", evaluation._verify_program_snippet("list", program_snippet))

Generated program from GPT:
def append(head: Node, val: int) -> None:
    """Appends a new node with the given value to the end of the list."""
    Requires(is_list(head))
    Ensures(is_list(head))
    Unfold(is_list(head))
    if head.next is None:
        n = Node(val)
        Fold(is_list(n))
        head.next = n
    else:
        Unfold(is_list(head))
        append(head.next, val)
        Fold(is_list(head))
response ['', 'Verification failed', 'Errors:', 'Unfold might fail. There might be insufficient permission to access is_list(head). (tmp.py@31.8)', 'Verification took 6.29 seconds.']
Verification result:
 Verification failed: Unfold might fail. There might be insufficient permission to access is_list(head). at line 11.8


### Run all
Repeat the above experiment for all examples with one example held out in each call to GPT and summarize the results.

In [25]:
import data
examples = data.get_list_of_examples("list")
print("Running Evaluation on examples:\n", examples)

Running Evaluation on examples:
 ['prepend', 'append', 'find', 'find_iter', 'remove', 'join_lists', 'reverse']


In [8]:
evaluation.run_eval(k=3, n=3, system_prompt="system_prompt_multi_round.txt")

Running example: prepend ; attempt: 1 ; error depth: 1
Generated program from GPT:
def prepend(head: Node, val: int) -> Node:
    """Prepends a new node with the given value to the list."""
    Requires(is_list(head))
    Ensures(is_list(Result()))

    n = Node(val, head)
    Fold(is_list(n))
    return n
response ['', 'Verification successful', 'Verification took 2.47 seconds.']
Verification result:
 Verification successful 


Running example: append ; attempt: 1 ; error depth: 1
Generated program from GPT:
def append(head: Node, val: int) -> None:
    """Appends a new node with the given value to the end of the list."""
    Requires(is_list(head))
    
    Unfold(is_list(head))
    if head.next is None:
        n = Node(val)
        head.next = n
        Fold(is_list(head))
    else:
        Fold(is_list(head))
        append(head.next, val)
response ['', 'Verification failed', 'Errors:', 'Method call might fail. There might be insufficient permission to access head.next. (tmp.py@32

EvalResult(results={'prepend': True, 'append': False, 'find': True, 'find_iter': False, 'remove': True, 'join_lists': True, 'reverse': False}, verified_at={'prepend': (1, 1), 'find': (2, 1), 'remove': (3, 2), 'join_lists': (3, 1)})

In [9]:
eval_result_gpt3_5 = _

In [11]:
eval_result_gpt4 = evaluation.run_eval(k=5, n=10, model="gpt-4", with_errors=True, system_prompt="system_prompt_multi_round.txt")

Running example: prepend ; attempt: 1 ; error depth: 1
Generated program from GPT:
def prepend(head: Node, val: int) -> Node:
    """Prepends a new node with the given value to the list."""
    Requires(is_list(head))
    Ensures(is_list(Result()) and Result().val == val and Result().next == head)
    Fold(is_list(head))
    n = Node(val, head)
    Unfold(is_list(head))
    return n

response ['', 'Verification failed', 'Errors:', 'Contract might not be well-formed. There might be insufficient permission to access Result().val. (tmp.py@24.12)', 'Verification took 3.16 seconds.']
Verification result:
 Verification failed: Contract might not be well-formed. There might be insufficient permission to access Result().val. at line 4.12 


Running example: prepend ; attempt: 1 ; error depth: 2
Generated program from GPT:
def prepend(head: Node, val: int) -> Node:
    """Prepends a new node with the given value to the list."""
    Requires(is_list(head))
    Ensures(is_list(Result()))
    Fold

In [2]:
eval_result_gpt4

NameError: name 'eval_result_gpt4' is not defined

In [3]:
import numpy as np
K,N = 5,10
verified_at_plot:np.ndarray = np.zeros((K,N), dtype=int)

for _, val in eval_result_gpt3_5.verified_at.items():
    k,n = val
    verified_at_plot[k-1][n-1] += 1
#cumsum each row of verified_at_plot
verified_at_plot = verified_at_plot.cumsum(axis=1)
#cumsum each column of verified_at_plot
verified_at_plot = verified_at_plot.cumsum(axis=0)
NUM_EXAMPLES = len(examples)
import matplotlib.pyplot as plt

# plt.title("Cumulative Verified Examples")
plt.xlabel("Error depth (n)")
plt.ylabel("Number of Verified Examples")
plt.plot(np.arange(1, N+1), verified_at_plot[0], 'o')
plt.plot(np.arange(1, N+1), verified_at_plot[1], 'o')
plt.plot(np.arange(1, N+1), verified_at_plot[4], 'o')
plt.legend(["k=1", "k=2", "k=5"])
#show a horizontal line at y=NUM_EXAMPLES
plt.hlines(NUM_EXAMPLES, 1, N, colors='k', linestyles='dashed')
#show all integers 1..N on the x axis
plt.xticks(np.arange(1, N+1))
plt.show()

NameError: name 'eval_result_gpt3_5' is not defined